<H6 align="center">Белорусский государственный университет </H6>
<H6 align="center">Механико-математический факультет </H6>
<H6 align="center">Кафедра дифференциальных уравнений  и системного анализа </H6>

<H2 align="center">КУРСОВАЯ РАБОТА</H2>


<H2 align="center">Распознавание жестов</H2>

<H4 align="center">Габбасов Илья </H4>

## 1. Создание набора данных.

In [2]:
# импортируем необходимые библиотеки
import os
import cv2

In [2]:
# задаем путь к директории, где будут храниться данные
DATA = './data'

In [ ]:
# проверяем существование директории и создаем ее, если она не существует
if not os.path.exists(DATA):
    os.makedirs(DATA)

# задаем количество жестов, которые будут распознаваться
num_of_gestures = 32

# задаем количество изображений, которые будут собраны для каждого жеста
dataset_size = 200

# создаем объект VideoCapture для получения видеопотока
cap = cv2.VideoCapture(0)

# проходим по всем жестам и создаем для каждого жеста отдельную папку
for i in range(num_of_gestures):
    if not os.path.exists(os.path.join(DATA, str(i))):
        os.makedirs(os.path.join(DATA, str(i)))
        
# выводим сообщение о том, какой жест собираем в данный момент
    print('%%% Сбор данных для жеста под номером {} %%%'.format(i))
    
# ждем, пока пользователь не нажмет кнопку 'q', чтобы начать сбор данных для данного жеста
    while True:
        ret, frame = cap.read()
        frame = cv2.flip(frame, 1)
        cv2.putText(frame, ' Готовы? Для создания нажмите Q ! ', (10, 50), cv2.FONT_HERSHEY_COMPLEX, 0.9, (255, 0, 0), 2, cv2.LINE_AA)
        cv2.imshow('frame', frame)
        if cv2.waitKey(25) == ord('q'):
            break
            
# начинаем сбор данных для данного жеста            
    counter = 0
    
    while counter < dataset_size:
        ret, frame = cap.read()
        frame = cv2.flip(frame, 1)
        cv2.imshow('frame', frame)
        cv2.waitKey(25)
        
# сохраняем изображение с именем '{номер_жеста}_{номер_изображения}.jpg'
        cv2.imwrite(os.path.join(DATA, str(i), '{}.jpg'.format(counter)), frame)
        counter += 1
        
# освобождаем поток объекта VideoCapture и закрываем все окна        
cap.release()
cv2.destroyAllWindows()

## 2. Отслеживание кистей рук.

In [3]:
# импортируем необходимые библиотеки
import mediapipe as mp
import matplotlib.pyplot as plt

# используем модель для обнаружения и отслеживания рук
mp_hands=mp.solutions.hands

# используем модуль, содержащий стили для рисования
mp_drawing=mp.solutions.drawing_utils
mp_drawing_styles=mp.solutions.drawing_styles

# cоздаем объект-экземпляр модели для обнаружения и отслеживания рук
# на cтатическом изображении.
hands=mp_hands.Hands(static_image_mode=True,min_detection_confidence=0.6)

In [ ]:
# создаем список папок в директории
folders = os.listdir(DATA)

# создаем сетку графиков размером 7x5
fig, axs = plt.subplots(7, 5, figsize=(10, 10))

# проходим по каждой папке и выводим по одному изображению в каждой строке
for i, folder in enumerate(folders):
    
    # получаем список файлов в папке
    files = os.listdir(os.path.join(DATA, folder))
    if len(files) == 0:
        continue
        
    # выбираем первый файл
    img_path = os.path.join(DATA, folder, files[0])
    
    # загружаем изображение и конвертируем в RGB
    img = cv2.imread(img_path)
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
    # обрабатываем изображение и рисуем на нем ориентиры рук
    results = hands.process(img_rgb)
    for hand_landmarks in results.multi_hand_landmarks:
        mp_drawing.draw_landmarks(
            img_rgb,
            hand_landmarks,
            mp_hands.HAND_CONNECTIONS,
            mp_drawing_styles.get_default_hand_landmarks_style(),
            mp_drawing_styles.get_default_hand_connections_style())
        
    # выводим изображение в сетке графиков
    row = i // 5
    col = i % 5
    axs[row][col].imshow(img_rgb)
    axs[row][col].set_title(folder)
    axs[row][col].axis('off')

# удаляем лишние ячейки сетки, если они есть
for i in range(len(folders), 35):
    row = i // 5
    col = i % 5
    fig.delaxes(axs[row][col])

# показываем график
plt.tight_layout()
plt.show()

## 3. Создание обучающего набора.

In [ ]:
# создаем списки для хранения извлеченных данных о руках и метках
data=[]
labels=[]

# проходим по каждой папке и файлу в текущей директории
for dir_ in os.listdir(DATA):
    for img_path in os.listdir(os.path.join(DATA,dir_)):
        data_aux=[]
        img=cv2.imread(os.path.join(DATA,dir_,img_path))
        img_rgb=cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
        results=hands.process(img_rgb)
        if results.multi_hand_landmarks:
            # если обнаружены ориентиры рук на изображении
            for hand_landmarks in results.multi_hand_landmarks:
                # перебираем каждый ориентир руки на изображении
                for i in range(len(hand_landmarks.landmark)):
                    # вычисляем координаты ориентира
                    x=hand_landmarks.landmark[i].x
                    y=hand_landmarks.landmark[i].y
                    data_aux.append(x)
                    data_aux.append(y)
                    
            # добавляем данные о руке       
            data.append(data_aux)
            
            # добавляем метку для текущего изображения
            labels.append(dir_)

In [6]:
# импортируем необходимую библиотеку
import pickle

# осуществляем запись в файл
f=open('data.pickle','wb')
pickle.dump({'data':data,'labels':labels},f)
f.close()

## 4. Обучение.

In [7]:
# загрузка обучающего набора данных
data_dict=pickle.load(open('./data.pickle1','rb'))

In [8]:
# импортируем классы и функций из библиотеки scikit-learn 
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [17]:
# импортируем необходимую библиотеку
import numpy as np

# преобразуем данные в массивы numpy
data=np.asarray(data_dict['data'])
labels=np.asarray(data_dict['labels'])

# разделяем данные на обучающую и тестовую выборки
x_train,x_test,y_train,y_test=train_test_split(data,labels,
                test_size=0.2,shuffle=True,stratify=labels)

In [18]:
# создаем экземпляр классификатора
model=RandomForestClassifier()

# обучаем модель
model.fit(x_train,y_train)

# применяем модель к тестовым данным
y_predict= model.predict(x_test)

# рассчитываем точность классификации
score = accuracy_score(y_predict,y_test)
print('{}% образцов было классифицированы корректно!'.format(score*100))

99.6012759170654% образцов было классифицированы корректно!


In [19]:
# осуществляем запись в файл
f=open('model.p','wb')
pickle.dump({'model':model},f)
f.close()

## 5. Тестирование.

In [6]:
# импортируем необходимые библиотеки
import os
import cv2
import mediapipe as mp
import matplotlib.pyplot as plt
import pickle
import numpy as np

# cоздаем объект-экземпляр модели
# для обнаружения и отслеживания рук
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5,
    max_num_hands=1) 

In [7]:
# загрузка обученной модели
model_dict = pickle.load(open('./model.p', 'rb'))
model = model_dict['model']

# создаем объект VideoCapture для получения видеопотока
cap = cv2.VideoCapture(0)

# создаем алфавит соответсвующий меткам
labels_dict = labels_dict = {0:'А', 1:'Б', 2:'В',
3:'Г', 4:'Д', 5:'Е', 6:'Ж', 7:'З', 8:'И', 9:'Й',
10:'К', 11:'Л', 12:'М', 13:'Н', 14:'О', 15:'П', 
16:'Р', 17:'С', 18:'Т', 19:'У', 20:'Ф', 21:'Х',
22:'Ц', 23:'Ч', 24:'Ш', 25:'Щ', 26:'Ь', 27:'Ъ',
28:'Ы', 29:'Э', 30:'Ю', 31:'Я'}

# обрабатываем кадры видеопотока
while True:
    data_aux = []
    x_, y_ = [], []

    ret, frame = cap.read()
    H, W, _ = frame.shape
    frame = cv2.flip(frame, 1)
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    results = hands.process(frame_rgb)

    if results.multi_hand_landmarks:
        # если обнаружены ориентиры рук на изображении
        for hand_landmarks in results.multi_hand_landmarks:
            # перебираем каждый ориентир руки на изображении
            mp_drawing.draw_landmarks(
                frame,
                hand_landmarks,
                mp_hands.HAND_CONNECTIONS,
                mp_drawing_styles.get_default_hand_landmarks_style(),
                mp_drawing_styles.get_default_hand_connections_style())

            for i in range(len(hand_landmarks.landmark)):
                # вычисляем координаты ориентира
                x = hand_landmarks.landmark[i].x
                y = hand_landmarks.landmark[i].y
                data_aux.append(x)
                data_aux.append(y)
                x_.append(x)
                y_.append(y)
        
        # вычисляем границы рамки
        x1 = int(min(x_) * W) - 20
        y1 = int(min(y_) * H) - 20

        x2 = int(max(x_) * W) + 90
        y2 = int(max(y_) * H) + 10
        
        # получаем резултат распознавания
        prediction = model.predict([data_aux])
        predicted_letter = labels_dict[int(prediction[0])]
        
        # отображаем результат
        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 0, 0), 3)
        cv2.putText(frame, predicted_letter, (x1, y1 - 10),
        cv2.FONT_HERSHEY_COMPLEX, 1.3, (255, 0, 0), 2, cv2.LINE_AA)
        
    # выход из программы
    cv2.imshow('frame', frame)
    if cv2.waitKey(1) == ord('q'):
        break
        
# освобождаем поток объекта VideoCapture и закрываем все окна 
cap.release()
cv2.destroyAllWindows()

## 6. Продвинутая версия.

In [4]:
# импортируем необходимые библиотеки
import os
import cv2
import mediapipe as mp
import matplotlib.pyplot as plt
import pickle
import numpy as np

# cоздаем объект-экземпляр модели
# для обнаружения и отслеживания рук
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5,
    max_num_hands=1) 

In [5]:
# загрузка обученной модели
import time

model_dict = pickle.load(open('./model.p', 'rb'))
model = model_dict['model']

# создаем объект VideoCapture для получения видеопотока
cap = cv2.VideoCapture(0)

# создаем алфавит соответсвующий меткам
labels_dict = labels_dict = {0:'А', 1:'Б', 2:'В',
3:'Г', 4:'Д', 5:'Е', 6:'Ж', 7:'З', 8:'И', 9:'Й',
10:'К', 11:'Л', 12:'М', 13:'Н', 14:'О', 15:'П', 
16:'Р', 17:'С', 18:'Т', 19:'У', 20:'Ф', 21:'Х',
22:'Ц', 23:'Ч', 24:'Ш', 25:'Щ', 26:'Ь', 27:'Ъ',
28:'Ы', 29:'Э', 30:'Ю', 31:'Я'}

current_letter = ''
last_prediction_time = 0
last_letter_time = 0
sentence = ''

# обрабатываем кадры видеопотока
while True:
    data_aux = []
    x_, y_ = [], []

    ret, frame = cap.read()
    H, W, _ = frame.shape
    frame = cv2.flip(frame, 1)
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    results = hands.process(frame_rgb)

    if results.multi_hand_landmarks:
         # если обнаружены ориентиры рук на изображении
        for hand_landmarks in results.multi_hand_landmarks:
            # перебираем каждый ориентир руки на изображении
            mp_drawing.draw_landmarks(
                frame,
                hand_landmarks,
                mp_hands.HAND_CONNECTIONS,
                mp_drawing_styles.get_default_hand_landmarks_style(),
                mp_drawing_styles.get_default_hand_connections_style())

        for hand_landmarks in results.multi_hand_landmarks:
            for i in range(len(hand_landmarks.landmark)):
                # вычисляем координаты ориентира
                x = hand_landmarks.landmark[i].x
                y = hand_landmarks.landmark[i].y
                data_aux.append(x)
                data_aux.append(y)
                x_.append(x)
                y_.append(y)
                
        # вычисляем границы рамки
        x1 = int(min(x_) * W) - 20
        y1 = int(min(y_) * H) - 20

        x2 = int(max(x_) * W) +90
        y2 = int(max(y_) * H) +10
        
        # набор текста с помощью жестов
        current_time = time.time()
        if current_time - last_prediction_time >= 1:
            prediction = model.predict([data_aux])
            predicted_letter = labels_dict[int(prediction[0])]
            if predicted_letter == current_letter:
                if current_time - last_letter_time >= 1:
                    sentence += current_letter
                    last_letter_time = current_time
                    cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 165, 255), -1)
            else:
                current_letter = predicted_letter
                last_letter_time = current_time
                last_prediction_time = current_time
       
                
        # отображаем результат
        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 0, 0), 3)
        cv2.putText(frame, current_letter, (x1, y1 - 10), cv2.FONT_HERSHEY_COMPLEX, 1.3, (255, 0, 0), 2, cv2.LINE_AA)
    cv2.putText(frame, sentence, (20, 50), cv2.FONT_HERSHEY_COMPLEX, 1, (255, 0, 0), 2, cv2.LINE_AA)
    cv2.imshow('frame', frame)
    
    # выход из программы
    if cv2.waitKey(1) == ord('q'):
        break
        
# освобождаем поток объекта VideoCapture и закрываем все окна 
cap.release()
cv2.destroyAllWindows()